In [1]:
import pandas as pd
import requests
import os

In [2]:
# 1. Carencia de material en familias.
url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/60147?tip=AM&"
response = requests.get(url)
data_json = response.json()

registros = []

# Recorremos cada indicador en el JSON y lo guardamos en registros.
for indicador in data_json:
    tipo_carencia = next(meta['Nombre'] for meta in indicador['MetaData'] if meta['T3_Variable'] == 'Tipo de carencia de material')
    regimen_tenencia = next(meta['Nombre'] for meta in indicador['MetaData'] if meta['T3_Variable'] == 'Régimen de tenenecia de la vivienda principal')

    for registro in indicador['Data']:
        registros.append({
            'Tipo_Carencia': tipo_carencia,
            'Regimen_Tenencia': regimen_tenencia,
            'Anyo': registro['Anyo'],
            'Valor': registro['Valor']
        })

df = pd.DataFrame(registros)


# Análisis inicial del df. Vemos la cabecera, los tipos de datos, nulos, etc.
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos de tipo de carencia: ")
print(df['Tipo_Carencia'].unique())

print("\nValores únicos de regimen de tenencia: ")
print(df['Regimen_Tenencia'].unique())

# Una vez realizado el análisis inicial, pasamos a la limpieza de datos.
# Tratamiento de nulos y espacios en blanco.
df['Tipo_Carencia'] = df['Tipo_Carencia'].fillna('Desconocido').str.strip()
df['Regimen_Tenencia'] = df['Regimen_Tenencia'].fillna('Desconocido').str.strip()

# Conversión de tipos de datos y optamos por eliminar los valores nulos de la columna principal de análisis.
df['Anyo'] = df['Anyo'].astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

# Mostramos el dataset tras las transformaciones.
print(df.info())
print(df.head())

# Creación de las tablas de dimensiones y la tabla de hechos para el modelo relacional de power bi.
dim_tipo_carencia_material = pd.DataFrame({
    'id_tipo_carencia_material': range(1, len(df['Tipo_Carencia'].unique()) + 1),
    'Tipo_Carencia': df['Tipo_Carencia'].unique()
})

dim_regimen_tenencia_carencia_material = pd.DataFrame({
    'id_regimen_tenencia_carencia_material': range(1, len(df['Regimen_Tenencia'].unique()) + 1),
    'Regimen_Tenencia': df['Regimen_Tenencia'].unique()
})

fact_carencia_material = df.merge(dim_tipo_carencia_material, on='Tipo_Carencia') \
                           .merge(dim_regimen_tenencia_carencia_material, on='Regimen_Tenencia')

fact_carencia_material = fact_carencia_material[['id_tipo_carencia_material', 'id_regimen_tenencia_carencia_material', 'Anyo', 'Valor']]

# Definimos las carpetas de salida
base_path = 'Personas con carencia material por régimen de tenencia de la vivienda'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

# Creamos las carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos CSV
fact_carencia_material.to_csv(os.path.join(fact_path, 'fact_carencia_material.csv'), index=False)
dim_tipo_carencia_material.to_csv(os.path.join(dim_path, 'dim_tipo_carencia_material.csv'), index=False)
dim_regimen_tenencia_carencia_material.to_csv(os.path.join(dim_path, 'dim_regimen_tenencia_carencia_material.csv'), index=False)

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
                                       Tipo_Carencia Regimen_Tenencia  Anyo  \
0  No puede permitirse ir de vacaciones al menos ...            Total  2024   
1  No puede permitirse ir de vacaciones al menos ...            Total  2023   
2  No puede permitirse ir de vacaciones al menos ...            Total  2022   
3  No puede permitirse ir de vacaciones al menos ...            Total  2021   
4  No puede permitirse ir de vacaciones al menos ...            Total  2020   

   Valor  
0   33.4  
1   33.1  
2   33.5  
3   32.7  
4   34.4  

Tipos de datos originales:
Tipo_Carencia        object
Regimen_Tenencia     object
Anyo                  int64
Valor               float64
dtype: object

Valores nulos por columna:
Tipo_Carencia        0
Regimen_Tenencia     0
Anyo                 0
Valor               62
dtype: int64

Valores únicos de tipo de carencia: 
['No puede permitirse ir de vacaciones al menos una semana al año'
 'No puede permitirse una comida de 

In [3]:
# 2. Personas por decil de renta por unidad de consumo y régimen de tenencia de la vivienda

# Descargamos los datos desde la API del INE
url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/59953?tip=AM&"
response = requests.get(url)
data_json = response.json()

registros = []

# Recorremos el JSON y extraemos los datos relevantes
for indicador in data_json:
    try:
        decil_renta = next(meta['Nombre'] for meta in indicador['MetaData'] if meta['T3_Variable'] == 'Porcentajes de renta por unidad de consumo')
        regimen_tenencia = next(meta['Nombre'] for meta in indicador['MetaData'] if meta['T3_Variable'] == 'Régimen de tenenecia de la vivienda principal')
    except StopIteration:
        continue

    for registro in indicador['Data']:
        registros.append({
            'Decil_Renta': decil_renta,
            'Regimen_Tenencia': regimen_tenencia,
            'Anyo': registro['Anyo'],
            'Valor': registro['Valor']
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Decil_Renta':")
print(df['Decil_Renta'].unique())

print("\nValores únicos en 'Regimen_Tenencia':")
print(df['Regimen_Tenencia'].unique())

# Limpieza y transformación de los datos
df['Decil_Renta'] = df['Decil_Renta'].fillna('Desconocido').str.strip()
df['Regimen_Tenencia'] = df['Regimen_Tenencia'].fillna('Desconocido').str.strip()
df['Anyo'] = df['Anyo'].astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las tablas de dimensiones y la tabla de hechos
dim_decil_renta = pd.DataFrame({
    'id_decil_renta': range(1, len(df['Decil_Renta'].unique()) + 1),
    'Decil_Renta': df['Decil_Renta'].unique()
})

dim_regimen_tenencia_decil_renta = pd.DataFrame({
    'id_regimen_tenencia_decil_renta': range(1, len(df['Regimen_Tenencia'].unique()) + 1),
    'Regimen_Tenencia': df['Regimen_Tenencia'].unique()
})

fact_personas_decil_renta = df.merge(dim_decil_renta, on='Decil_Renta') \
                              .merge(dim_regimen_tenencia_decil_renta, on='Regimen_Tenencia') \
                              [['id_decil_renta', 'id_regimen_tenencia_decil_renta', 'Anyo', 'Valor']]

# Definimos las carpetas de salida
base_path = 'Personas por decil de renta por unidad de consumo y régimen de tenencia de la vivienda'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

# Creamos las carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos CSV
fact_personas_decil_renta.to_csv(os.path.join(fact_path, 'fact_personas_decil_renta.csv'), index=False)
dim_decil_renta.to_csv(os.path.join(dim_path, 'dim_decil_renta.csv'), index=False)
dim_regimen_tenencia_decil_renta.to_csv(os.path.join(dim_path, 'dim_regimen_tenencia_decil_renta.csv'), index=False)

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
    Decil_Renta Regimen_Tenencia  Anyo  Valor
0  Primer decil            Total  2024   10.0
1  Primer decil            Total  2023   10.0
2  Primer decil            Total  2022   10.0
3  Primer decil            Total  2021   10.0
4  Primer decil            Total  2020   10.0

Tipos de datos originales:
Decil_Renta          object
Regimen_Tenencia     object
Anyo                  int64
Valor               float64
dtype: object

Valores nulos por columna:
Decil_Renta          0
Regimen_Tenencia     0
Anyo                 0
Valor               40
dtype: int64

Valores únicos en 'Decil_Renta':
['Primer decil' 'Segundo decil' 'Tercer decil' 'Cuarto decil'
 'Quinto decil' 'Sexto decil' 'Septimo decil' 'Octavo decil'
 'Noveno decil' 'Décimo decil']

Valores únicos en 'Regimen_Tenencia':
['Total' 'Propiedad' 'Propiedad sin hipoteca' 'Propiedad con hipoteca'
 'Alquiler a precio de mercado' 'Alquiler inferior al precio de mercado'
 'Cesión']
<class 'pandas.core.fra

In [5]:
# 3. Renta por hogar por régimen de tenencia de la vivienda

url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/59956?tip=AM&"
response = requests.get(url)
data_json = response.json()

registros = []

# Recorremos el JSON y extraemos los datos relevantes
for item in data_json:
    tipo_renta = next(meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'SALDOS CONTABLES')
    regimen_tenencia = next(meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Régimen de tenenecia de la vivienda principal')

    for registro in item['Data']:
        registros.append({
            'Tipo_Renta': tipo_renta,
            'Regimen_Tenencia': regimen_tenencia,
            'Anyo': registro['Anyo'],
            'Valor': registro['Valor']
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Tipo_Renta':")
print(df['Tipo_Renta'].unique())

print("\nValores únicos en 'Regimen_Tenencia':")
print(df['Regimen_Tenencia'].unique())

# Limpieza y transformación de los datos
df['Tipo_Renta'] = df['Tipo_Renta'].fillna('Desconocido').str.strip()
df['Regimen_Tenencia'] = df['Regimen_Tenencia'].fillna('Desconocido').str.strip()
df['Anyo'] = df['Anyo'].astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las tablas de dimensiones y la tabla de hechos
dim_tipo_renta_hogar = pd.DataFrame({
    'id_tipo_renta_hogar': range(1, len(df['Tipo_Renta'].unique()) + 1),
    'Tipo_Renta': df['Tipo_Renta'].unique()
})

dim_regimen_tenencia_renta_hogar = pd.DataFrame({
    'id_regimen_tenencia_renta_hogar': range(1, len(df['Regimen_Tenencia'].unique()) + 1),
    'Regimen_Tenencia': df['Regimen_Tenencia'].unique()
})

fact_renta_hogar = df.merge(dim_tipo_renta_hogar, on='Tipo_Renta') \
                     .merge(dim_regimen_tenencia_renta_hogar, on='Regimen_Tenencia') \
                     [['id_tipo_renta_hogar', 'id_regimen_tenencia_renta_hogar', 'Anyo', 'Valor']]

# Definimos las carpetas de salida
base_path = "Renta por hogar por régimen de tenencia de la vivienda"
fact_path = os.path.join(base_path, "fact")
dim_path = os.path.join(base_path, "dims")

# Creamos las carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos CSV
fact_renta_hogar.to_csv(os.path.join(fact_path, "fact_renta_hogar.csv"), index=False)
dim_tipo_renta_hogar.to_csv(os.path.join(dim_path, "dim_tipo_renta_hogar.csv"), index=False)
dim_regimen_tenencia_renta_hogar.to_csv(os.path.join(dim_path, "dim_regimen_tenencia_renta_hogar.csv"), index=False)

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
                   Tipo_Renta Regimen_Tenencia  Anyo    Valor
0  Renta neta media por hogar            Total  2024  36996.0
1  Renta neta media por hogar            Total  2023  34821.0
2  Renta neta media por hogar            Total  2022  32216.0
3  Renta neta media por hogar            Total  2021  30552.0
4  Renta neta media por hogar            Total  2020  30690.0

Tipos de datos originales:
Tipo_Renta           object
Regimen_Tenencia     object
Anyo                  int64
Valor               float64
dtype: object

Valores nulos por columna:
Tipo_Renta          0
Regimen_Tenencia    0
Anyo                0
Valor               8
dtype: int64

Valores únicos en 'Tipo_Renta':
['Renta neta media por hogar'
 'Renta media por hogar (con alquiler imputado)']

Valores únicos en 'Regimen_Tenencia':
['Total' 'Propiedad' 'Propiedad sin hipoteca' 'Propiedad con hipoteca'
 'Alquiler a precio de mercado' 'Alquiler inferior al precio de mercado'
 'Cesión']
<class 

In [6]:
# 4. Renta por persona y unidad consumo por grado de urbanización

import pandas as pd
import requests
import os

# Descargamos los datos desde la API del INE
url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/59947?tip=AM&"
response = requests.get(url)
data_json = response.json()

registros = []

# Recorremos el JSON y extraemos los datos relevantes
for item in data_json:
    tipo_renta = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'SALDOS CONTABLES'), 'Desconocido')
    grado_urb = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Grado de urbanización'), 'Desconocido')

    for registro in item['Data']:
        registros.append({
            'Tipo_Renta': tipo_renta,
            'Grado_Urbanizacion': grado_urb,
            'Anyo': registro['Anyo'],
            'Valor': registro['Valor']
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Tipo_Renta':")
print(df['Tipo_Renta'].unique())

print("\nValores únicos en 'Grado_Urbanizacion':")
print(df['Grado_Urbanizacion'].unique())

# Limpieza y transformación de los datos
df['Tipo_Renta'] = df['Tipo_Renta'].fillna('Desconocido').str.strip()
df['Grado_Urbanizacion'] = df['Grado_Urbanizacion'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las tablas de dimensiones y la tabla de hechos
dim_tipo_renta_urbanizacion = pd.DataFrame({
    'id_tipo_renta_urbanizacion': range(1, len(df['Tipo_Renta'].unique()) + 1),
    'Tipo_Renta': df['Tipo_Renta'].unique()
})

dim_grado_urbanizacion_renta = pd.DataFrame({
    'id_grado_urbanizacion_renta': range(1, len(df['Grado_Urbanizacion'].unique()) + 1),
    'Grado_Urbanizacion': df['Grado_Urbanizacion'].unique()
})

fact_renta_urbanizacion = df.merge(dim_tipo_renta_urbanizacion, on='Tipo_Renta') \
                            .merge(dim_grado_urbanizacion_renta, on='Grado_Urbanizacion') \
                            [['id_tipo_renta_urbanizacion', 'id_grado_urbanizacion_renta', 'Anyo', 'Valor']]

# Definimos las rutas de salida
base_path = "Renta por persona y unidad consumo por grado de urbanización"
fact_path = os.path.join(base_path, "fact")
dim_path = os.path.join(base_path, "dims")

# Creamos las carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos CSV
fact_renta_urbanizacion.to_csv(os.path.join(fact_path, "fact_renta_urbanizacion.csv"), index=False, encoding='utf-8')
dim_tipo_renta_urbanizacion.to_csv(os.path.join(dim_path, "dim_tipo_renta_urbanizacion.csv"), index=False, encoding='utf-8')
dim_grado_urbanizacion_renta.to_csv(os.path.join(dim_path, "dim_grado_urbanizacion_renta.csv"), index=False, encoding='utf-8')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
                     Tipo_Renta Grado_Urbanizacion  Anyo    Valor
0  Renta neta media por persona              Total  2024  14807.0
1  Renta neta media por persona              Total  2023  14082.0
2  Renta neta media por persona              Total  2022  13008.0
3  Renta neta media por persona              Total  2021  12269.0
4  Renta neta media por persona              Total  2020  12292.0

Tipos de datos originales:
Tipo_Renta             object
Grado_Urbanizacion     object
Anyo                    int64
Valor                 float64
dtype: object

Valores nulos por columna:
Tipo_Renta            0
Grado_Urbanizacion    0
Anyo                  0
Valor                 0
dtype: int64

Valores únicos en 'Tipo_Renta':
['Renta neta media por persona' 'Renta media por unidad de consumo'
 'Renta media por persona (con alquiler imputado)'
 'Renta media por unidad de consumo (con alquiler imputado)']

Valores únicos en 'Grado_Urbanizacion':
['Total' 'Área dens

In [7]:
# 5. Renta por persona y unidad consumo por régimen de tenencia de la vivienda

url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/59949?tip=AM&"
response = requests.get(url)
data_json = response.json()

registros = []

# Recorremos cada entrada del JSON y extraemos las variables relevantes
for item in data_json:
    tipo_renta = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'SALDOS CONTABLES'), 'Desconocido')
    categoria = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Régimen de tenenecia de la vivienda principal'), 'Desconocido')

    for registro in item['Data']:
        registros.append({
            'Tipo_Renta': tipo_renta,
            'Categoria': categoria,
            'Anyo': registro['Anyo'],
            'Valor': registro['Valor']
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Tipo_Renta':")
print(df['Tipo_Renta'].unique())

print("\nValores únicos en 'Categoria':")
print(df['Categoria'].unique())

# Limpieza y transformación de los datos
df['Tipo_Renta'] = df['Tipo_Renta'].fillna('Desconocido').str.strip()
df['Categoria'] = df['Categoria'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las tablas de dimensiones y la tabla de hechos
dim_tipo_renta_tenencia = pd.DataFrame({
    'id_tipo_renta_tenencia': range(1, len(df['Tipo_Renta'].unique()) + 1),
    'Tipo_Renta': df['Tipo_Renta'].unique()
})

dim_categoria_tenencia = pd.DataFrame({
    'id_categoria_tenencia': range(1, len(df['Categoria'].unique()) + 1),
    'Categoria': df['Categoria'].unique()
})

fact_personas_renta_tenencia = df.merge(dim_tipo_renta_tenencia, on='Tipo_Renta') \
                                 .merge(dim_categoria_tenencia, on='Categoria') \
                                 [['id_tipo_renta_tenencia', 'id_categoria_tenencia', 'Anyo', 'Valor']]

# Definimos rutas de carpetas
base_path = "Renta por persona y unidad consumo por régimen de tenencia de la vivienda"
fact_path = os.path.join(base_path, "fact")
dim_path = os.path.join(base_path, "dims")

# Creamos las carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos CSV
fact_personas_renta_tenencia.to_csv(os.path.join(fact_path, "fact_personas_renta_tenencia.csv"), index=False, encoding='utf-8')
dim_tipo_renta_tenencia.to_csv(os.path.join(dim_path, "dim_tipo_renta_tenencia.csv"), index=False, encoding='utf-8')
dim_categoria_tenencia.to_csv(os.path.join(dim_path, "dim_categoria_tenencia.csv"), index=False, encoding='utf-8')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
                     Tipo_Renta Categoria  Anyo    Valor
0  Renta neta media por persona     Total  2024  14807.0
1  Renta neta media por persona     Total  2023  14082.0
2  Renta neta media por persona     Total  2022  13008.0
3  Renta neta media por persona     Total  2021  12269.0
4  Renta neta media por persona     Total  2020  12292.0

Tipos de datos originales:
Tipo_Renta     object
Categoria      object
Anyo            int64
Valor         float64
dtype: object

Valores nulos por columna:
Tipo_Renta     0
Categoria      0
Anyo           0
Valor         16
dtype: int64

Valores únicos en 'Tipo_Renta':
['Renta neta media por persona' 'Renta media por unidad de consumo'
 'Renta media por persona (con alquiler imputado)'
 'Renta media por unidad de consumo (con alquiler imputado)']

Valores únicos en 'Categoria':
['Total' 'Propiedad' 'Propiedad sin hipoteca' 'Propiedad con hipoteca'
 'Alquiler a precio de mercado' 'Alquiler inferior al precio de mercado

In [8]:
# 6. Renta por persona y unidad de consumo por tamaño de hogar

url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/59945?tip=AM"
response = requests.get(url)
data_json = response.json()

# Recorremos cada entrada del JSON y almacenamos los registros
registros = []

for item in data_json:
    tipo_renta = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'SALDOS CONTABLES'), 'Desconocido')
    tamano_hogar = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Composición del hogar'), 'Desconocido')

    for dato in item['Data']:
        registros.append({
            'Tipo_Renta': tipo_renta,
            'Tamano_Hogar': tamano_hogar,
            'Anyo': dato['Anyo'],
            'Valor': dato.get('Valor', None)
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Tipo_Renta':")
print(df['Tipo_Renta'].unique())

print("\nValores únicos en 'Tamano_Hogar':")
print(df['Tamano_Hogar'].unique())

# Limpieza y transformación de los datos
df['Tipo_Renta'] = df['Tipo_Renta'].fillna('Desconocido').str.strip()
df['Tamano_Hogar'] = df['Tamano_Hogar'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las tablas de dimensiones y la tabla de hechos
dim_tipo_renta_tamanohogar = pd.DataFrame({
    'id_tipo_renta_tamanohogar': range(1, len(df['Tipo_Renta'].unique()) + 1),
    'Tipo_Renta': df['Tipo_Renta'].unique()
})

dim_tamano_hogar_renta = pd.DataFrame({
    'id_tamano_hogar_renta': range(1, len(df['Tamano_Hogar'].unique()) + 1),
    'Tamano_Hogar': df['Tamano_Hogar'].unique()
})

fact_personas_renta_tamanohogar = df.merge(dim_tipo_renta_tamanohogar, on='Tipo_Renta') \
                                    .merge(dim_tamano_hogar_renta, on='Tamano_Hogar') \
                                    [['id_tipo_renta_tamanohogar', 'id_tamano_hogar_renta', 'Anyo', 'Valor']]

# Definimos rutas de carpetas
base_path = 'Renta por persona y unidad de consumo por tamaño de hogar'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

# Creamos las carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos
fact_personas_renta_tamanohogar.to_csv(os.path.join(fact_path, 'fact_personas_renta_tamanohogar.csv'), index=False, encoding='utf-8')
dim_tipo_renta_tamanohogar.to_csv(os.path.join(dim_path, 'dim_tipo_renta_tamanohogar.csv'), index=False, encoding='utf-8')
dim_tamano_hogar_renta.to_csv(os.path.join(dim_path, 'dim_tamano_hogar_renta.csv'), index=False, encoding='utf-8')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
                     Tipo_Renta Tamano_Hogar  Anyo    Valor
0  Renta neta media por persona        Total  2024  14807.0
1  Renta neta media por persona        Total  2023  14082.0
2  Renta neta media por persona        Total  2022  13008.0
3  Renta neta media por persona        Total  2021  12269.0
4  Renta neta media por persona        Total  2020  12292.0

Tipos de datos originales:
Tipo_Renta       object
Tamano_Hogar     object
Anyo              int64
Valor           float64
dtype: object

Valores nulos por columna:
Tipo_Renta      0
Tamano_Hogar    0
Anyo            0
Valor           0
dtype: int64

Valores únicos en 'Tipo_Renta':
['Renta neta media por persona' 'Renta media por unidad de consumo'
 'Renta media por persona (con alquiler imputado)'
 'Renta media por unidad de consumo (con alquiler imputado)']

Valores únicos en 'Tamano_Hogar':
['Total' '1 miembro' '2 miembros' '3 miembros' '4 miembros'
 '5 miembros o más']
<class 'pandas.core.frame.Da

In [9]:
# 7. Renta por persona y unidad de consumo por tipo de hogar

url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/9946?tip=AM"
response = requests.get(url)
data_json = response.json()

# Recorremos cada entrada del JSON y almacenamos los registros
registros = []

for item in data_json:
    tipo_renta = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'SALDOS CONTABLES'), 'Desconocido')
    tipo_hogar = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Tipo de hogar'), 'Desconocido')

    for dato in item['Data']:
        registros.append({
            'Tipo_Renta': tipo_renta,
            'Tipo_Hogar': tipo_hogar,
            'Anyo': dato['Anyo'],
            'Valor': dato.get('Valor', None)
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Tipo_Renta':")
print(df['Tipo_Renta'].unique())

print("\nValores únicos en 'Tipo_Hogar':")
print(df['Tipo_Hogar'].unique())

# Limpieza y transformación de los datos
df['Tipo_Renta'] = df['Tipo_Renta'].fillna('Desconocido').str.strip()
df['Tipo_Hogar'] = df['Tipo_Hogar'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las tablas de dimensiones y la tabla de hechos
dim_tipo_renta_tipohogar = pd.DataFrame({
    'id_tipo_renta_tipohogar': range(1, len(df['Tipo_Renta'].unique()) + 1),
    'Tipo_Renta': df['Tipo_Renta'].unique()
})

dim_tipo_hogar_renta = pd.DataFrame({
    'id_tipo_hogar_renta': range(1, len(df['Tipo_Hogar'].unique()) + 1),
    'Tipo_Hogar': df['Tipo_Hogar'].unique()
})

fact_personas_renta_tipohogar = df.merge(dim_tipo_renta_tipohogar, on='Tipo_Renta') \
                                  .merge(dim_tipo_hogar_renta, on='Tipo_Hogar') \
                                  [['id_tipo_renta_tipohogar', 'id_tipo_hogar_renta', 'Anyo', 'Valor']]

# Definimos carpetas
base_path = 'Renta por persona y unidad de consumo por tipo de hogar'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

# Creamos carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos
fact_personas_renta_tipohogar.to_csv(os.path.join(fact_path, 'fact_personas_renta_tipohogar.csv'), index=False, encoding='utf-8')
dim_tipo_renta_tipohogar.to_csv(os.path.join(dim_path, 'dim_tipo_renta_tipohogar.csv'), index=False, encoding='utf-8')
dim_tipo_hogar_renta.to_csv(os.path.join(dim_path, 'dim_tipo_hogar_renta.csv'), index=False, encoding='utf-8')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
                     Tipo_Renta Tipo_Hogar  Anyo    Valor
0  Renta neta media por persona      Total  2024  14807.0
1  Renta neta media por persona      Total  2023  14082.0
2  Renta neta media por persona      Total  2022  13008.0
3  Renta neta media por persona      Total  2021  12269.0
4  Renta neta media por persona      Total  2020  12292.0

Tipos de datos originales:
Tipo_Renta     object
Tipo_Hogar     object
Anyo            int64
Valor         float64
dtype: object

Valores nulos por columna:
Tipo_Renta    0
Tipo_Hogar    0
Anyo          0
Valor         4
dtype: int64

Valores únicos en 'Tipo_Renta':
['Renta neta media por persona' 'Renta media por unidad de consumo'
 'Renta media por persona (con alquiler imputado)'
 'Renta media por unidad de consumo (con alquiler imputado)']

Valores únicos en 'Tipo_Hogar':
['Total' 'Hogares de una persona' '2 adultos sin niños dependientes'
 'Otros hogares sin niños dependientes'
 '1 adulto con 1 ó más niños d

In [10]:
# 8. Tasa de riesgo de pobreza por régimen de tenencia de la vivienda

url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/59960?tip=AM"
response = requests.get(url)
data_json = response.json()

# Recorremos cada entrada del JSON y extraemos los campos necesarios
registros = []

for item in data_json:
    concepto_pobreza = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Conceptos Demográficos'), 'Desconocido')
    regimen_vivienda = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Régimen de tenenecia de la vivienda principal'), 'Desconocido')

    for dato in item['Data']:
        registros.append({
            'Concepto_Pobreza': concepto_pobreza,
            'Regimen_Vivienda': regimen_vivienda,
            'Anyo': dato['Anyo'],
            'Valor': dato.get('Valor', None)
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Concepto_Pobreza':")
print(df['Concepto_Pobreza'].unique())

print("\nValores únicos en 'Regimen_Vivienda':")
print(df['Regimen_Vivienda'].unique())

# Limpieza y transformación de los datos
df['Concepto_Pobreza'] = df['Concepto_Pobreza'].fillna('Desconocido').str.strip()
df['Regimen_Vivienda'] = df['Regimen_Vivienda'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las tablas de dimensiones y la tabla de hechos
dim_concepto_pobreza = pd.DataFrame({
    'id_concepto_pobreza': range(1, len(df['Concepto_Pobreza'].unique()) + 1),
    'Concepto_Pobreza': df['Concepto_Pobreza'].unique()
})

dim_regimen_vivienda_riesgo = pd.DataFrame({
    'id_regimen_vivienda_riesgo': range(1, len(df['Regimen_Vivienda'].unique()) + 1),
    'Regimen_Vivienda': df['Regimen_Vivienda'].unique()
})

fact_tasa_riesgo_pobreza = df.merge(dim_concepto_pobreza, on='Concepto_Pobreza') \
                             .merge(dim_regimen_vivienda_riesgo, on='Regimen_Vivienda') \
                             [['id_concepto_pobreza', 'id_regimen_vivienda_riesgo', 'Anyo', 'Valor']]

# Definimos rutas
base_path = 'Tasa de riesgo de pobreza por régimen de tenencia de la vivienda'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

# Creamos las carpetas si no existen
os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

# Guardamos los archivos CSV
fact_tasa_riesgo_pobreza.to_csv(os.path.join(fact_path, 'fact_tasa_riesgo_pobreza.csv'), index=False, encoding='utf-8')
dim_concepto_pobreza.to_csv(os.path.join(dim_path, 'dim_concepto_pobreza.csv'), index=False, encoding='utf-8')
dim_regimen_vivienda_riesgo.to_csv(os.path.join(dim_path, 'dim_regimen_vivienda_riesgo.csv'), index=False, encoding='utf-8')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
                                    Concepto_Pobreza Regimen_Vivienda  Anyo  \
0  Tasa de riesgo de pobreza (renta del año anter...            Total  2024   
1  Tasa de riesgo de pobreza (renta del año anter...            Total  2023   
2  Tasa de riesgo de pobreza (renta del año anter...            Total  2022   
3  Tasa de riesgo de pobreza (renta del año anter...            Total  2021   
4  Tasa de riesgo de pobreza (renta del año anter...            Total  2020   

   Valor  
0   19.7  
1   20.2  
2   20.4  
3   21.7  
4   21.0  

Tipos de datos originales:
Concepto_Pobreza     object
Regimen_Vivienda     object
Anyo                  int64
Valor               float64
dtype: object

Valores nulos por columna:
Concepto_Pobreza    0
Regimen_Vivienda    0
Anyo                0
Valor               8
dtype: int64

Valores únicos en 'Concepto_Pobreza':
['Tasa de riesgo de pobreza (renta del año anterior a la entrevista)'
 'Tasa de riesgo de pobreza (con alq

In [11]:
# 9. Renta por hogar y comunidad autonoma

import pandas as pd
import requests
import os

# Descargamos los datos desde la API del INE
url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/9947?tip=AM&"
response = requests.get(url)
data_json = response.json()

# Recorremos el JSON y almacenamos los registros excluyendo el total nacional
registros = []

for item in data_json:
    comunidad = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] in ['Comunidades y Ciudades Autónomas', 'Totales Territoriales']), 'Desconocido')
    tipo_renta = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'SALDOS CONTABLES'), 'Desconocido')

    if comunidad.strip() == "Total Nacional":
        continue

    for dato in item['Data']:
        registros.append({
            'Comunidad_Autonoma': comunidad,
            'Tipo_Renta': tipo_renta,
            'Anyo': dato['Anyo'],
            'Valor': dato.get('Valor', None)
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Comunidad_Autonoma':")
print(df['Comunidad_Autonoma'].unique())

print("\nValores únicos en 'Tipo_Renta':")
print(df['Tipo_Renta'].unique())

# Limpieza de datos
df['Comunidad_Autonoma'] = df['Comunidad_Autonoma'].fillna('Desconocido').str.strip()
df['Tipo_Renta'] = df['Tipo_Renta'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las dimensiones y la tabla de hechos
dim_comunidad = pd.DataFrame({
    'id_comunidad': range(1, len(df['Comunidad_Autonoma'].unique()) + 1),
    'Comunidad_Autonoma': df['Comunidad_Autonoma'].unique()
})

dim_tipo_renta = pd.DataFrame({
    'id_tipo_renta': range(1, len(df['Tipo_Renta'].unique()) + 1),
    'Tipo_Renta': df['Tipo_Renta'].unique()
})

fact_renta_comunidades = df.merge(dim_comunidad, on='Comunidad_Autonoma') \
                           .merge(dim_tipo_renta, on='Tipo_Renta') \
                           [['id_comunidad', 'id_tipo_renta', 'Anyo', 'Valor']]

# =====================================
# 6. Guardamos los archivos en carpetas organizadas
# =====================================
base_path = 'Renta por hogar por comunidades autónomas'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

fact_renta_comunidades.to_csv(os.path.join(fact_path, 'fact_renta_comunidades.csv'), index=False, encoding='utf-8-sig')
dim_comunidad.to_csv(os.path.join(dim_path, 'dim_comunidad.csv'), index=False, encoding='utf-8-sig')
dim_tipo_renta.to_csv(os.path.join(dim_path, 'dim_tipo_renta_comunidades.csv'), index=False, encoding='utf-8-sig')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
  Comunidad_Autonoma                    Tipo_Renta  Anyo    Valor
0          Andalucía  Renta neta media por persona  2024  12191.0
1          Andalucía  Renta neta media por persona  2023  11719.0
2          Andalucía  Renta neta media por persona  2022  10703.0
3          Andalucía  Renta neta media por persona  2021   9915.0
4          Andalucía  Renta neta media por persona  2020   9990.0

Tipos de datos originales:
Comunidad_Autonoma     object
Tipo_Renta             object
Anyo                    int64
Valor                 float64
dtype: object

Valores nulos por columna:
Comunidad_Autonoma    0
Tipo_Renta            0
Anyo                  0
Valor                 0
dtype: int64

Valores únicos en 'Comunidad_Autonoma':
['Andalucía' 'Aragón' 'Asturias, Principado de' 'Balears, Illes'
 'Canarias' 'Cantabria' 'Castilla y León' 'Castilla - La Mancha'
 'Cataluña' 'Comunitat Valenciana' 'Extremadura' 'Galicia'
 'Madrid, Comunidad de' 'Murcia, Región de' 

In [13]:
# 10. Riesgo de pobreza por comunidades autonomas

url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/9963?tip=AM&"
response = requests.get(url)
data_json = response.json()

# Recorremos el JSON y almacenamos los registros
registros = []

for item in data_json:
    comunidad = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] in ['Totales Territoriales', 'Comunidades y Ciudades Autónomas']), 'Desconocido')
    concepto = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Conceptos Demográficos'), 'Desconocido')

    if comunidad.strip() == "Total Nacional":
        continue

    for dato in item['Data']:
        registros.append({
            'Comunidad_Autonoma': comunidad,
            'Concepto_Pobreza': concepto,
            'Anyo': dato['Anyo'],
            'Valor': dato.get('Valor', None)
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Comunidad_Autonoma':")
print(df['Comunidad_Autonoma'].unique())

print("\nValores únicos en 'Concepto_Pobreza':")
print(df['Concepto_Pobreza'].unique())

# Limpieza de datos
df['Comunidad_Autonoma'] = df['Comunidad_Autonoma'].fillna('Desconocido').str.strip()
df['Concepto_Pobreza'] = df['Concepto_Pobreza'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las dimensiones y la tabla de hechos
dim_comunidad_pobreza = pd.DataFrame({
    'id_comunidad_pobreza': range(1, len(df['Comunidad_Autonoma'].unique()) + 1),
    'Comunidad_Autonoma': df['Comunidad_Autonoma'].unique()
})

dim_concepto_pobreza = pd.DataFrame({
    'id_concepto_pobreza': range(1, len(df['Concepto_Pobreza'].unique()) + 1),
    'Concepto_Pobreza': df['Concepto_Pobreza'].unique()
})

fact_pobreza_comunidades = df.merge(dim_comunidad_pobreza, on='Comunidad_Autonoma') \
                              .merge(dim_concepto_pobreza, on='Concepto_Pobreza') \
                              [['id_comunidad_pobreza', 'id_concepto_pobreza', 'Anyo', 'Valor']]

# =====================================
# 6. Guardamos los archivos en carpetas organizadas
# =====================================
base_path = 'Tasa de riesgo de pobreza por comunidades autónomas'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

fact_pobreza_comunidades.to_csv(os.path.join(fact_path, 'fact_pobreza_comunidades.csv'), index=False, encoding='utf-8')
dim_comunidad_pobreza.to_csv(os.path.join(dim_path, 'dim_comunidad_pobreza.csv'), index=False, encoding='utf-8')
dim_concepto_pobreza.to_csv(os.path.join(dim_path, 'dim_concepto_pobreza.csv'), index=False, encoding='utf-8')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
  Comunidad_Autonoma                                   Concepto_Pobreza  Anyo  \
0          Andalucía  Tasa de riesgo de pobreza (renta del año anter...  2024   
1          Andalucía  Tasa de riesgo de pobreza (renta del año anter...  2023   
2          Andalucía  Tasa de riesgo de pobreza (renta del año anter...  2022   
3          Andalucía  Tasa de riesgo de pobreza (renta del año anter...  2021   
4          Andalucía  Tasa de riesgo de pobreza (renta del año anter...  2020   

   Valor  
0   29.2  
1   30.5  
2   29.1  
3   32.3  
4   28.5  

Tipos de datos originales:
Comunidad_Autonoma     object
Concepto_Pobreza       object
Anyo                    int64
Valor                 float64
dtype: object

Valores nulos por columna:
Comunidad_Autonoma    0
Concepto_Pobreza      0
Anyo                  0
Valor                 0
dtype: int64

Valores únicos en 'Comunidad_Autonoma':
['Andalucía' 'Aragón' 'Asturias, Principado de' 'Balears, Illes'
 'Canarias'

In [14]:
# 11. Regimen de tenencia por comunidad autonoma

import pandas as pd
import requests
import os

# Descargamos los datos desde la API del INE
url = "https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/9997?tip=AM&"
response = requests.get(url)
data_json = response.json()

# Procesamos los datos en una tabla
registros = []

for item in data_json:
    comunidad = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] in ['Totales Territoriales', 'Comunidades y Ciudades Autónomas']), 'Desconocido')
    regimen = next((meta['Nombre'] for meta in item['MetaData'] if meta['T3_Variable'] == 'Régimen de tenenecia de la vivienda principal'), 'Desconocido')
    
    if comunidad.strip() == "Total Nacional":
        continue

    for dato in item['Data']:
        registros.append({
            'Comunidad_Autonoma': comunidad,
            'Regimen_Tenencia': regimen,
            'Anyo': dato['Anyo'],
            'Valor': dato.get('Valor', None)
        })

df = pd.DataFrame(registros)

# Análisis inicial del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())

print("\nTipos de datos originales:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nValores únicos en 'Comunidad_Autonoma':")
print(df['Comunidad_Autonoma'].unique())

print("\nValores únicos en 'Regimen_Tenencia':")
print(df['Regimen_Tenencia'].unique())

# Limpieza de datos
df['Comunidad_Autonoma'] = df['Comunidad_Autonoma'].fillna('Desconocido').str.strip()
df['Regimen_Tenencia'] = df['Regimen_Tenencia'].fillna('Desconocido').str.strip()
df['Anyo'] = pd.to_numeric(df['Anyo'], errors='coerce').astype(int)
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
df = df.dropna(subset=['Valor']).reset_index(drop=True)

print(df.info())
print(df.head())

# Creamos las dimensiones y la tabla de hechos
dim_comunidad_hogar = pd.DataFrame({
    'id_comunidad_hogar': range(1, len(df['Comunidad_Autonoma'].unique()) + 1),
    'Comunidad_Autonoma': df['Comunidad_Autonoma'].unique()
})

dim_regimen_hogar = pd.DataFrame({
    'id_regimen_hogar': range(1, len(df['Regimen_Tenencia'].unique()) + 1),
    'Regimen_Tenencia': df['Regimen_Tenencia'].unique()
})

fact_hogares_tenencia_comunidad = df.merge(dim_comunidad_hogar, on='Comunidad_Autonoma') \
                                     .merge(dim_regimen_hogar, on='Regimen_Tenencia') \
                                     [['id_comunidad_hogar', 'id_regimen_hogar', 'Anyo', 'Valor']]

# =====================================
# 6. Guardamos los archivos en carpetas organizadas
# =====================================
base_path = 'Hogares por régimen de tenencia de la vivienda y comunidades autónomas'
fact_path = os.path.join(base_path, 'fact')
dim_path = os.path.join(base_path, 'dims')

os.makedirs(fact_path, exist_ok=True)
os.makedirs(dim_path, exist_ok=True)

fact_hogares_tenencia_comunidad.to_csv(os.path.join(fact_path, 'fact_hogares_tenencia_comunidad.csv'), index=False, encoding='utf-8')
dim_comunidad_hogar.to_csv(os.path.join(dim_path, 'dim_comunidad_hogar.csv'), index=False, encoding='utf-8')
dim_regimen_hogar.to_csv(os.path.join(dim_path, 'dim_regimen_hogar.csv'), index=False, encoding='utf-8')

print("\nArchivos guardados correctamente en:", base_path)


Primeras filas del DataFrame:
  Comunidad_Autonoma Regimen_Tenencia  Anyo  Valor
0          Andalucía        Propiedad  2024   77.9
1          Andalucía        Propiedad  2023   77.6
2          Andalucía        Propiedad  2022   78.8
3          Andalucía        Propiedad  2021   78.3
4          Andalucía        Propiedad  2020   78.4

Tipos de datos originales:
Comunidad_Autonoma     object
Regimen_Tenencia       object
Anyo                    int64
Valor                 float64
dtype: object

Valores nulos por columna:
Comunidad_Autonoma    0
Regimen_Tenencia      0
Anyo                  0
Valor                 1
dtype: int64

Valores únicos en 'Comunidad_Autonoma':
['Andalucía' 'Aragón' 'Asturias, Principado de' 'Balears, Illes'
 'Canarias' 'Cantabria' 'Castilla y León' 'Castilla - La Mancha'
 'Cataluña' 'Comunitat Valenciana' 'Extremadura' 'Galicia'
 'Madrid, Comunidad de' 'Murcia, Región de' 'Navarra, Comunidad Foral de'
 'País Vasco' 'Rioja, La' 'Ceuta' 'Melilla']

Valores únicos